## 文章复现：特质风险、投资者偏好与股票收益———基于前景理论视角的分析

### 理论部分

本文在前景理论的视角下分析特质风险、投资者偏好与股票收益之间的关系，首先建立了理论模型探究特质风险对投资者偏好的影响，并通过蒙特卡洛模拟的方法得出如下结论: 当股票有未实现的资本损失时，特质波动率越大的股票对应的前景理论效用值越大，因此投资者会偏好高特质波动率的股票; 当股票有未实现的资本收益时，特质波动率越小的股票对应的前景理论效用值越高，投资者会偏好低特质波动率的股票． 
![title](png/2.png)

在此基础上，对整体市场的“特质波动率之谜”做出了理论解释: 对于有未实现的资本损失的股票，投资者偏好行为会导致特质波动率与预期收益率呈负相关关系，而对于有未实现的资本收益的股票，投资者偏好行为会导致特质波动率与预期收益率呈正相关关系，由于资本市场上普遍存在的处置效应，投资者倾向于卖出盈利的股票而继续持有亏损的股票，因此整体市场上有未实现的资本损失的股票对应的特质波动率与预期收益率之间的负相关关系占据主导地位，所以整体股票市场中存在“特质波动率之谜”．

理论分析的结论可以归纳成以下两个假说，将在后文中使用分组检验和Fama-MacBeth 回归的方法对此进行实证检验:

**假说1：对于有未实现的资本损失( CL) 的股票，预期收益率与特质波动率负相关; 对于有未实现的收益资本收益( CG) 的股票，预期收益率与特质波动率正相关．**

**假说2：整体市场而言，“特质波动率之谜”存在，即预期收益率与特质波动率之间呈负相关**

最后，从变量替换与参数设定的角度验证了结论的稳健性．

### 实证检验部分

本文以中国沪深两市全部A 股( 包括中小板和创业板) 作为实证研究对象，样本期选择从2001年1月4日～ 2015年12月31日． 股票的交易数据和财务数据来自CSMAR金融数据库，无风险收益率数据来自锐思金融研究数据库．

实证检验中的被解释变量为股票的前复权收益率( Return)，核心解释变量及分组依据变量包括特质波动率(IVOL) 、未实现的资本利得(CGO) 、前景理论价值(PT) 和期望效用理论价值(EU) ，控制变量包括流通市值(Size) 、账面价值与市值之比(BM) 、换手率(Turnover) 、动量效应(MOM) 、偏度系数(Skew).

In [1]:
import pandas as pd
import numpy as np
import random
import os
from scipy import stats
from scipy.stats.mstats import winsorize
from datetime import datetime
from functools import reduce
from pandas.tseries.offsets import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.formula.api as sml

首先将股票月内全部日收益率进行如下的五因子回归算出IVOL和CGO变量:

$\begin{aligned} R_{i, d}-R_{f, d}=& \alpha_{i}+\beta_{i} M K T_{d}+s_{i} S M B_{d}+ h_{i} H M L_{d}+c_{i} C M A_{d}+r_{i} R M W_{d}+\varepsilon_{i, d} \end{aligned}$

以回归式中的常数项作为股票未实现的资本利得CGO，以衡量股票当前收益率相对于其参考点基准收益率的偏离程度。

回归方程残差项的标准差即为股票的特质波动率IVOL。

------------------
首先从导入数据中筛选年度交易数据和财务数据完整的股票

In [77]:
# 月度财务变量合并
df=pd.read_csv('TRD_Mnth.csv')

# 筛选股票:中国沪深两市全部A 股(包括中小板和创业板)
df=df[(df['Markettype'].isin([1,4,16]))]
df['Month']=pd.to_datetime(df['Month'])
df['Size']=df['Size']*1000  # 单位换算
del df['Markettype']

# 将月末数据改为月初方便合并
def monthbegin(name,var):
    temp=pd.read_csv(name)
    temp[var]=temp[var].apply(lambda x:str(x)[:-3]) # 将账面市值比的月末数据改为月初方便合并
    temp[var]=pd.to_datetime(temp[var])
    return temp

# 合并账面市值比数据
BM = monthbegin('FI_T10.csv','Month')

# 合并净利润数据
RMW = monthbegin('FI_T5.csv','Month')
RMW = RMW[RMW['Typrep']=='A']
del RMW['Typrep']

# 合并资产增长率数据
CMA = monthbegin('FI_T8.csv','Month')
CMA=CMA[CMA['Typrep']=='A']
del CMA['Typrep']

# 合并换手率数据
Turnover=pd.read_csv('LIQ_TOVER_M.csv')
Turnover['Month']=pd.to_datetime(Turnover['Month'])

monthly_frame = \
    reduce(lambda left, right: pd.merge(left, right, on=['Month','Stkcd'],how='left'), [df,Turnover,BM,RMW,CMA])

In [78]:
# 对月度数据进行排序
monthly_frame.sort_values(['Month','Stkcd'],ascending=True,inplace=True)

# 对季度(年度)数据BM、CMA、RMW进行以年度为单位的向前月度填充
monthly_frame['year']=monthly_frame['Month'].dt.year
def yfill(var):
    monthly_frame[var]=monthly_frame.groupby(['Stkcd'])[var].shift(1) #以去年年末数据作为今年年初数据
    monthly_frame[var]=monthly_frame.groupby(['Stkcd','year'])[var].ffill() 
    return monthly_frame[var]

monthly_frame['BM']=yfill('BM')
monthly_frame['CMA']=yfill('CMA')
monthly_frame['RMW']=yfill('RMW')

# 将月度换手率换算成小数
monthly_frame['Turnover']=monthly_frame['Turnover']*0.01

In [80]:
# 删除仍然存在的缺失值
monthly_frame.dropna(inplace=True)

In [81]:
monthly_frame

,Stkcd,Month,Size,Return,Turnover,BM,RMW,CMA,year
46729,563,1999-07-01,1.352052e+09,-0.133603,0.139139,0.641203,0.062352,0.030001,1999
46730,563,1999-08-01,1.376060e+09,0.017757,0.068411,0.641203,0.062352,0.030001,1999
46731,563,1999-09-01,1.353316e+09,-0.016529,0.129477,0.641203,0.062352,0.030001,1999
46732,563,1999-10-01,1.291399e+09,-0.045752,0.031059,0.641203,0.062352,0.030001,1999
46733,563,1999-11-01,1.267391e+09,-0.018591,0.065475,0.641203,0.062352,0.030001,1999
...,...,...,...,...,...,...,...,...,...
579228,605388,2021-06-01,1.593200e+09,-0.073667,2.690524,0.707131,0.032283,0.014759,2021
579232,605389,2021-06-01,1.427884e+09,0.001002,1.686787,0.608542,0.043287,0.944797,2021
579238,605398,2021-06-01,6.501667e+08,-0.027153,1.033783,0.702129,0.011476,0.688946,2021
579249,605399,2021-06-01,1.081460e+09,0.296031,1.580063,0.729846,0.052381,0.056246,2021


In [92]:
# 与月度无风险收益合并
Rf=pd.read_csv('rf.csv')
Rf['Month']=pd.to_datetime(Rf['Month'])
alldata=pd.merge(monthly_frame, Rf, on=['Month'],how='right')

alldata.dropna(subset=['Stkcd'],inplace=True)
alldata.sort_values(['Stkcd','Month'],ascending=True,inplace=True)

现已保证年度内公司财务数据完整，接下来筛选日度交易数据

因为日度数据文件太大，我没有上传到github，感兴趣的同学可以自行到

In [202]:
# 日度交易数据导入并拼接
os.chdir('C:/Users/win9zhanhuaxiong/Desktop/特质风险、投资者偏好与股票收益 - 副本/初始数据/daydata')
file_chdir = os.getcwd()
 
filecsv_list = []
for root,dirs,files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            filecsv_list.append(file)
 
data = pd.DataFrame()
for csv in filecsv_list:
    data = data.append(pd.read_csv(csv,header = 0,sep=None,encoding='gb18030'))

E:\anaconda\lib\site-packages\ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  del sys.path[0]


In [203]:
# Dretwd [考虑现金红利再投资的日个股回报率] ,Dretnd [不考虑现金红利的日个股回报率] 
data.rename(columns={'锘縎tkcd': 'Stkcd', 'Trddt': 'date', 'Dsmvosd': 'M', 'Dretwd': 'Return'}, inplace=True)
data['date']=pd.to_datetime(data['date'])
data=data.sort_values(['date','Stkcd'])
data.head(10)

,Stkcd,date,M,Return,Markettype
0,1,2001-01-04,19991340.36,-0.006233,4
706,2,2001-01-04,5924858.49,0.001348,4
1412,3,2001-01-04,1470113.34,-0.027919,4
1504,4,2001-01-04,1347603.69,-0.028237,4
2213,5,2001-01-04,2696029.56,-0.018773,4
2925,6,2001-01-04,2166690.65,-0.006512,4
3637,7,2001-01-04,1115444.67,-0.014758,4
4338,8,2001-01-04,1764175.74,0.000600,4
5045,9,2001-01-04,4266789.62,-0.012081,4
5757,10,2001-01-04,691032.20,-0.004280,4


In [204]:
# 筛选股票:中国沪深两市全部A股(包括中小板和创业板)
os.chdir('C:\\Users\\win9zhanhuaxiong\\code_all\\实证定价\\特质风险、投资者偏好与股票收益')

df=data.copy()
df=df[(df['Markettype'].isin([1,4,16]))]
del df['Markettype']

# 与日度无风险利率数据合并
Rf=pd.read_csv('rf_day.csv')
Rf['date']=pd.to_datetime(Rf['date'])

daydata = \
    reduce(lambda left, right: pd.merge(left, right, on=['date'],how='left'), [df,Rf])

In [205]:
# 从alldata中导入日度五因子计算所需数据
daydata_1=alldata.loc[:,['Stkcd','Month','BM','RMW','CMA']]

daydata['Month']=pd.to_datetime(daydata['date'].apply(lambda x:str(x)[:7]))

#合并数据
ff5_pr=pd.merge(daydata,daydata_1,on=['Month','Stkcd'],how='inner')

In [206]:
ff5_pr[ff5_pr.isna().values==True]

,Stkcd,date,M,Return,rf,Month,BM,RMW,CMA


In [207]:
# 筛选出月内交易天数大于10天的数据
def ten(day):
    a=day.groupby(['Month','Stkcd'])['Return'].agg(['count']).reset_index()
    day=day.merge(a,on=('Month','Stkcd'),how='outer')
    day=day[day['count']>=10]
    return day
ff5_pr=ten(ff5_pr)

ff5_pr.rename(columns={'M': 'SMB', 'BM': 'HML'}, inplace=True)
del ff5_pr['count']

In [208]:
# 计算日度五因子值
def factor(var):
    df=ff5_pr.copy()
    q = [0,0.5,1]
    q1 =[0,0.3,0.7,1]
    df.loc[:,'SMB'] = df.groupby('date')['SMB'].apply(lambda x:pd.qcut(x,q,labels=False) ) #q分组以将连续变量离散化
    df.loc[:,var] = df.groupby(['date','SMB'])[var].apply(lambda x:pd.qcut(x,q1,labels=False))
    df.groupby(['date','SMB',var])['Return'].mean()

    a=df.groupby(['date','SMB',var])['Return'].agg(['mean']).reset_index()
    date = a[['date']].drop_duplicates()
    X = pd.DataFrame()
    for t in date['date']:
        b=a[a['date']==t]
        x=(b.iloc[5,3]+b.iloc[2,3]-b.iloc[0,3]-b.iloc[3,3])/2
        y=(b.iloc[0,3]+b.iloc[1,3]+b.iloc[2,3]-b.iloc[3,3]-b.iloc[4,3]-b.iloc[5,3])/3
        temp=pd.DataFrame({'date':t, var:x,'SMB':y},index=[0])
        X = pd.concat([X,temp],axis=0,join='outer',ignore_index=True)
    return X

In [213]:
#计算因子收益率结果
HML=factor('HML')
RMW=factor('RMW')
CMA=factor('CMA')

ff4 = \
    reduce(lambda left, right: pd.merge(left, right, on=['date'],how='left'), [HML,RMW,CMA])

ff4['SMB']=(ff4['SMB']+ff4['SMB_x']+ff4['SMB_y'])/3
del ff4['SMB_x']
del ff4['SMB_y']

# 计算市场因子与合并
ff5_pr['ER']=ff5_pr['Return']-ff5_pr['rf']
MKT=ff5_pr.groupby('date')['ER'].agg(['mean']).reset_index()
ff5 = \
    reduce(lambda left, right: pd.merge(left, right, on=['date'],how='inner'), [ff4,MKT])

ff5.rename(columns={'mean': 'MKT'}, inplace=True)
ff5

,date,HML,RMW,CMA,SMB,MKT
0,2001-01-04,0.002761,0.000296,0.000495,0.004143,-0.002069
1,2001-01-05,0.005382,0.003069,0.002423,-0.000526,0.004167
2,2001-01-08,0.011562,0.003077,0.000330,-0.004477,-0.012746
3,2001-01-09,0.003579,0.001048,-0.001258,-0.001290,-0.002877
4,2001-01-10,0.005724,-0.004001,-0.004650,0.000957,0.011759
...,...,...,...,...,...,...
3625,2015-12-25,0.002490,-0.004417,-0.005367,0.003971,0.008972
3626,2015-12-28,-0.004107,-0.001524,-0.000904,-0.001819,-0.025319
3627,2015-12-29,0.002727,-0.001203,-0.001225,0.003472,0.012880
3628,2015-12-30,-0.009488,-0.002730,-0.000767,0.004120,0.012258


In [216]:
ff5_pr.drop(['SMB','HML','RMW','CMA','Month'], axis=1, inplace=True)
FF5 = \
    reduce(lambda left, right: pd.merge(left, right, on=['date'],how='inner'), [ff5_pr,ff5])

In [217]:
FF5

,Stkcd,date,Return,rf,ER,HML,RMW,CMA,SMB,MKT
0,1,2001-01-04,-0.006233,0.000054,-0.006287,0.002761,0.000296,0.000495,0.004143,-0.002069
1,2,2001-01-04,0.001348,0.000054,0.001294,0.002761,0.000296,0.000495,0.004143,-0.002069
2,4,2001-01-04,-0.028237,0.000054,-0.028291,0.002761,0.000296,0.000495,0.004143,-0.002069
3,5,2001-01-04,-0.018773,0.000054,-0.018827,0.002761,0.000296,0.000495,0.004143,-0.002069
4,6,2001-01-04,-0.006512,0.000054,-0.006566,0.002761,0.000296,0.000495,0.004143,-0.002069
...,...,...,...,...,...,...,...,...,...,...
5816868,600777,2015-12-31,-0.011608,0.000085,-0.011693,0.005218,-0.002289,-0.005407,-0.006249,-0.018253
5816869,2655,2015-12-31,0.010656,0.000085,0.010571,0.005218,-0.002289,-0.005407,-0.006249,-0.018253
5816870,300016,2015-12-31,-0.046711,0.000085,-0.046796,0.005218,-0.002289,-0.005407,-0.006249,-0.018253
5816871,300061,2015-12-31,-0.063343,0.000085,-0.063428,0.005218,-0.002289,-0.005407,-0.006249,-0.018253


In [218]:
# 计算五因子等回归
FF5 = FF5.sort_values(['date','Stkcd'], ascending=True) # 重新排序
df=FF5.copy()

start_month = pd.to_datetime('1/01/2001')
end_month = pd.to_datetime('12/01/2015')
ff5_1 = pd.DataFrame()

while start_month  <= end_month:
    
    df_month = df[(df['date']>=start_month) & (df['date']<start_month+MonthBegin(1))]
    group = df_month.groupby("Stkcd")
    Stkcd_index = group.size().index.tolist() # 取出分组数据的index，并生成list
    temp = pd.DataFrame()
    
    # 循环index，取出相应的index对应的子数据集，然后计算五因子指标和三因子alpha、CAPMalpha
    for Stkcd in Stkcd_index:
        Stkcd_data = group.get_group(Stkcd).reset_index()
        independent_list = Stkcd_data[['MKT','SMB','HML','RMW','CMA']]
        model = sm.OLS(endog = Stkcd_data['ER'], exog = sm.add_constant(independent_list)).fit()
        
        independent_list_capm = Stkcd_data[['MKT']]
        model_capm = sm.OLS(endog = Stkcd_data['ER'], exog = sm.add_constant(independent_list_capm)).fit()
        
        independent_list_ff3 = Stkcd_data[['MKT','SMB','HML']]
        model_ff3 = sm.OLS(endog = Stkcd_data['ER'], exog = sm.add_constant(independent_list_ff3)).fit()

        temp1 = pd.DataFrame({'Month': start_month.strftime('%Y-%m'), 'Stkcd':Stkcd, 'IVOL': model.resid.std(), 'CGO': model.params[0], \
                             'CAPM Alpha': model_capm.params[0], 'FF3 Alpha': model_ff3.params[0]}, index=[0])
        temp = pd.concat([temp,temp1],axis=0,join='outer',ignore_index=True)
        
    ff5_1 = pd.concat([temp,ff5_1],axis=0,join='outer',ignore_index=True)
    start_month = start_month + MonthBegin(1)

In [219]:
ff5_1=ff5_1.sort_values(['Month','Stkcd']).reset_index()
ff5_1.drop(columns=['index'],inplace=True)
ff5_1

,Month,Stkcd,IVOL,CGO,CAPM Alpha,FF3 Alpha
0,2001-01,1,0.016960,0.005472,0.005799,0.005709
1,2001-01,2,0.008851,-0.003864,0.005263,-0.002597
2,2001-01,4,0.009275,-0.014603,-0.010162,-0.012253
3,2001-01,5,0.013726,-0.001315,0.007267,0.001015
4,2001-01,6,0.008545,-0.003719,0.003507,-0.003550
...,...,...,...,...,...,...
293077,2015-12,603988,0.024743,0.001760,0.001644,0.001169
293078,2015-12,603989,0.017113,-0.008221,-0.006773,-0.007133
293079,2015-12,603993,0.010244,-0.006877,-0.007565,-0.007648
293080,2015-12,603997,0.023149,-0.004345,-0.006837,-0.006503


将其他月度变量纳入到数据集中：公司规模、考虑现金红利再投资的月收益率、账面市值比、无风险利率、月度换手率等等

In [228]:
ff5_1['Month'] = pd.to_datetime(ff5_1['Month'])
alldata1=alldata.loc[:,['Stkcd','Month','Size','Return','Turnover','BM','rf']]
monthly_frame = \
    reduce(lambda left, right: pd.merge(left, right, on=['Month','Stkcd'],how='left'), [ff5_1,alldata1])

In [229]:
monthly_frame

,Month,Stkcd,IVOL,CGO,CAPM Alpha,FF3 Alpha,Size,Return,Turnover,BM,rf
0,2001-01-01,1,0.016960,0.005472,0.005799,0.005709,2.086901e+10,0.031680,0.049985,0.799729,0.001650
1,2001-01-01,2,0.008851,-0.003864,0.005263,-0.002597,5.952768e+09,0.067191,0.310419,0.599784,0.001650
2,2001-01-01,4,0.009275,-0.014603,-0.010162,-0.012253,1.166396e+09,-0.151515,0.074732,0.278788,0.001650
3,2001-01-01,5,0.013726,-0.001315,0.007267,0.001015,2.971135e+09,0.117723,0.195771,0.477119,0.001650
4,2001-01-01,6,0.008545,-0.003719,0.003507,-0.003550,2.163534e+09,0.037850,0.222251,0.744766,0.001650
...,...,...,...,...,...,...,...,...,...,...,...
293077,2015-12-01,603988,0.024743,0.001760,0.001644,0.001169,1.312864e+09,0.132560,2.600188,0.613850,0.002549
293078,2015-12-01,603989,0.017113,-0.008221,-0.006773,-0.007133,2.676750e+09,-0.069846,1.847389,0.582124,0.002549
293079,2015-12-01,603993,0.010244,-0.006877,-0.007565,-0.007648,5.777364e+10,-0.123772,0.166354,0.621725,0.002549
293080,2015-12-01,603997,0.023149,-0.004345,-0.006837,-0.006503,1.620600e+09,-0.065721,2.323128,0.653706,0.002549


mom: 股票的动量定义为除上个月外的此前12个月度累计收益率

skew: 股票的偏度系数定义为过去5年的月收益率的偏度

In [240]:
# 动量因子计算
   # 注意：计算mom时，滚动窗宽的时候加入groupy 对于stock 索引，所以需要mom.values 赋值
   # 先log化乘法为加法再exp
data2=monthly_frame.loc[:,['Stkcd','Month','Return']]
data2['Return']=(data2['Return']+1).apply(np.log)
mom = data2.groupby('Stkcd').apply(lambda x:x.Return.rolling(12).sum()).reset_index()
mom.Return=mom.groupby('Stkcd')['Return'].shift(1)
mom.Return=mom.Return.apply(np.exp)-1

alldata=monthly_frame.copy()
alldata=alldata.sort_values(['Stkcd','Month'])
alldata['mom']=mom.Return

# 股票偏度系数计算
skew= data2.groupby('Stkcd').apply(lambda x:x.Return.rolling(60).skew())
alldata['skew']=skew.values
alldata=alldata.sort_values(['Month','Stkcd'])

In [241]:
# 加入邵计算的PT、EU变量
alldata_1=pd.read_csv('data_final.csv')
alldata_1=alldata_1.loc[:,['Month','Stkcd','pt','eu']]
alldata_1['Month']=pd.to_datetime(alldata_1['Month'])

alldata=pd.merge(alldata,alldata_1,on=['Month','Stkcd'],how='inner')

In [243]:
alldata

,Month,Stkcd,IVOL,CGO,CAPM Alpha,FF3 Alpha,Size,Return,Turnover,BM,rf,mom,skew,pt,eu
0,2001-01-01,1,0.016960,0.005472,0.005799,0.005709,2.086901e+10,0.031680,0.049985,0.799729,0.001650,NaN,NaN,NaN,NaN
1,2001-01-01,2,0.008851,-0.003864,0.005263,-0.002597,5.952768e+09,0.067191,0.310419,0.599784,0.001650,NaN,NaN,NaN,NaN
2,2001-01-01,4,0.009275,-0.014603,-0.010162,-0.012253,1.166396e+09,-0.151515,0.074732,0.278788,0.001650,NaN,NaN,NaN,NaN
3,2001-01-01,5,0.013726,-0.001315,0.007267,0.001015,2.971135e+09,0.117723,0.195771,0.477119,0.001650,NaN,NaN,NaN,NaN
4,2001-01-01,6,0.008545,-0.003719,0.003507,-0.003550,2.163534e+09,0.037850,0.222251,0.744766,0.001650,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293077,2015-12-01,603988,0.024743,0.001760,0.001644,0.001169,1.312864e+09,0.132560,2.600188,0.613850,0.002549,NaN,NaN,NaN,NaN
293078,2015-12-01,603989,0.017113,-0.008221,-0.006773,-0.007133,2.676750e+09,-0.069846,1.847389,0.582124,0.002549,NaN,NaN,NaN,NaN
293079,2015-12-01,603993,0.010244,-0.006877,-0.007565,-0.007648,5.777364e+10,-0.123772,0.166354,0.621725,0.002549,NaN,NaN,NaN,NaN
293080,2015-12-01,603997,0.023149,-0.004345,-0.006837,-0.006503,1.620600e+09,-0.065721,2.323128,0.653706,0.002549,NaN,NaN,NaN,NaN


首先在每个月度时间截面上计算各个变量的均值、标准差和分位数，然后报告每个月度横截面统计量时间序列的均值．

In [245]:
#根据和各位学长学姐讨论所得，认定CGO为日度数据，乘以30化为月度指标
alldata['CGO']=alldata['CGO']*30
# 描述性统计
def describe(data,group,var):
    a=data.groupby([group])[var].describe().reset_index()
    result = pd.DataFrame({var:a.describe()['mean']})
    return result

temp = pd.DataFrame()
for var in alldata.columns[2:]:
    temp1=describe(alldata,'Month',var)
    temp = temp.join([temp1],how='outer')
    
Describe=temp.T
Describe.drop(columns=['count','min','max'],inplace =True)
Describe = Describe.applymap(lambda x:'%.4f' % x)

Describe

,mean,std,25%,50%,75%
IVOL,0.0163,0.0045,0.0134,0.0158,0.0193
CGO,0.0011,0.0031,-0.0002,0.0004,0.0018
CAPM Alpha,0.0000,0.0001,-0.0000,0.0000,0.0000
FF3 Alpha,0.0000,0.0001,-0.0000,0.0000,0.0000
Size,5267347428.0676,4384658622.8408,1162550745.2786,4880254688.9183,8194912017.8761
Return,0.0165,0.1017,-0.0540,0.0164,0.0764
Turnover,0.4900,0.2974,0.2806,0.4204,0.6289
BM,0.6670,0.1200,0.5718,0.6559,0.7459
rf,0.0026,0.0011,0.0016,0.0023,0.0036
mom,0.2681,0.1018,0.2081,0.2441,0.3009


![title](png/1.png)

$\begin{aligned} R_{i, d}-R_{f, d}=& \alpha_{i}+\beta_{i} M K T_{d}+s_{i} S M B_{d}+ h_{i} H M L_{d}+c_{i} C M A_{d}+r_{i} R M W_{d}+\varepsilon_{i, d} \end{aligned}$

**假说1的FM回归检验：对于有未实现的资本损失( CL) 的股票，预期收益率与特质波动率负相关; 对于有未实现的收益资本收益( CG) 的股票，预期收益率与特质波动率正相关．**

通过Fama-MacBeth 回归的方法从个股水平上对不同样本内特质波动率与预期收益率之间的关系进行检验:
\begin{equation}
R_{i, t+1}=\alpha_{0}+\alpha_{1} I V O L_{i, t}^{r a n}+\sum_{j=1}^{n} \beta_{j} \text { Control }_{j, i, t}^{\operatorname{ran}} \qquad (20)
\end{equation}

在回归检验之前，将全部解释变量进行规模化处理

In [246]:
# 变量规模化处理
alldata_copy=alldata.copy()
alldata_copy['Month'] = pd.to_datetime(alldata_copy['Month'])

start_date = pd.to_datetime('01/01/2001')
end_date = pd.to_datetime('12/01/2015')

temp=pd.DataFrame()

while start_date <= end_date:
    temp1=pd.DataFrame()
    
    for var in alldata_copy.columns[2:]:
        a=alldata_copy.loc[alldata_copy['Month']==start_date,:].copy()
        a.loc[:,'group'] = pd.qcut(a[var],10,labels=False,duplicates="drop") 
        temp2=pd.DataFrame()
        temp2.loc[:,var]=a.loc[:,'group']/9-0.5
        temp1=pd.concat([temp1,temp2],axis=1,join='outer')
        
    temp=pd.concat([temp,temp1], axis=0, join='outer', ignore_index=True)
    
    start_date=start_date+MonthBegin(1)

alldata_copy.loc[:,['IVOL','CGO','Size','Turnover','BM','mom','skew']]=temp.loc[:,['IVOL','CGO','Size','Turnover','BM','mom','skew']]

In [247]:
alldata_copy

,Month,Stkcd,IVOL,CGO,CAPM Alpha,FF3 Alpha,Size,Return,Turnover,BM,rf,mom,skew,pt,eu
0,2001-01-01,1,0.500000,0.500000,0.005799,0.005709,0.500000,0.031680,-0.500000,0.500000,0.001650,NaN,NaN,NaN,NaN
1,2001-01-01,2,-0.055556,-0.388889,0.005263,-0.002597,0.500000,0.067191,0.388889,0.166667,0.001650,NaN,NaN,NaN,NaN
2,2001-01-01,4,0.055556,-0.500000,-0.010162,-0.012253,-0.055556,-0.151515,-0.500000,-0.500000,0.001650,NaN,NaN,NaN,NaN
3,2001-01-01,5,0.388889,-0.166667,0.007267,0.001015,0.500000,0.117723,0.166667,-0.166667,0.001650,NaN,NaN,NaN,NaN
4,2001-01-01,6,-0.055556,-0.388889,0.003507,-0.003550,0.388889,0.037850,0.277778,0.388889,0.001650,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293077,2015-12-01,603988,0.166667,0.166667,0.001644,0.001169,-0.500000,0.132560,0.500000,0.166667,0.002549,NaN,NaN,NaN,NaN
293078,2015-12-01,603989,-0.166667,-0.500000,-0.006773,-0.007133,-0.500000,-0.069846,0.500000,0.166667,0.002549,NaN,NaN,NaN,NaN
293079,2015-12-01,603993,-0.500000,-0.500000,-0.007565,-0.007648,0.500000,-0.123772,-0.500000,0.166667,0.002549,NaN,NaN,NaN,NaN
293080,2015-12-01,603997,0.166667,-0.277778,-0.006837,-0.006503,-0.500000,-0.065721,0.500000,0.166667,0.002549,NaN,NaN,NaN,NaN


In [248]:
# FM回归
from linearmodels import FamaMacBeth

alldata_copy['Rt_1']=alldata_copy.sort_values(['Month'],ascending=True).groupby('Stkcd')['Return'].shift(-1)
alldata_copy.dropna(subset=['Rt_1'],inplace=True)

#按CGO正负分组
alldata_CL=alldata_copy.loc[alldata['CGO']<0,:]
alldata_CG=alldata_copy.loc[alldata['CGO']>=0,:]

def FM(data, dv=['Rt_1'], idv=['IVOL']):
    fmdata = data.set_index(['Stkcd','Month'])
    fm = FamaMacBeth(dependent = fmdata[dv],
                     exog =sm.add_constant(fmdata[idv]))
    res_fm = fm.fit(cov_type= 'kernel',debiased = False,bandwidth = 4)
    return res_fm


In [249]:
res_1=FM(alldata_CL, dv=['Rt_1'], idv=['IVOL'])
res_2=FM(alldata_CL, dv=['Rt_1'], idv=['IVOL','Size','BM'])

alldata_CL_1=alldata_CL.dropna(subset=['mom','skew'])
res_3=FM(alldata_CL_1, dv=['Rt_1'], idv=['IVOL','Size','BM','Turnover','mom','skew'])

res_4=FM(alldata_CG, dv=['Rt_1'], idv=['IVOL'])
res_5=FM(alldata_CG, dv=['Rt_1'], idv=['IVOL','Size','BM'])

alldata_CG_1=alldata_CG.dropna(subset=['mom','skew'])
res_6=FM(alldata_CG_1, dv=['Rt_1'], idv=['IVOL','Size','BM','Turnover','mom','skew'])

![title](png/3.png)

括号内的数字为使用 4 阶滞后的 Newey-West 标准误计算的 t 统计量，* 、＊＊和＊＊＊分别表示在 10% 、5% 和 1%

实证结果表明:1) 在CL 组中，无论是否加入控制变量， IVOL项的回归系数始终显著为负，这说明对于有未实现的资本损失的股票，股票的预期回报与特质风险负相关． 在前景理论框架下，处于CL 组的股票对应于投资者的风险偏好区域，承担额外的特质风险会获得相应的效用补偿，因此投资者此时将偏好高特质风险的股票，这种非理性的投资者偏好行为将会造成高特质风险的股票价格被高估，因此对于CL 组的股票而言，预期收益率与特质波动率呈负相关关系．

2)对于CG 组的股票而言，预期收益率与特质波动率呈负相关关系．与理论预期不相符。

猜想：CGO指标与原文相差过大，可能是由于日度回归时被解释变量收益率指标的不同导致的，原文中使用的是wind的前复权收益率，复现时使用的是csmar的考虑现金红利再投资的股票日收益，采用不考虑现金红利再投资的股票日收益再做一遍复现，结果还是不理想，又多次对样本进行不同方法的填充筛选，并使用学习班代码手动计算FM回归，仍然不理想。

----------------------
**假说2检验：整体市场而言，“特质波动率之谜”存在，即预期收益率与特质波动率之间呈负相关**

同样也可以利用Fama-MacBeth 回归的方法在个股水平上对假说2 进行检验． 除了根据回归式( 20) 检验整体样本的预期收益率与特质波动率的关系外，为了说明投资者偏好行为对股票收益率的影响，利用回归式( 21) 对前景理论效用值与预期收益率的关系进行检验． 此外，为了对比前
景理论与期望效用理论的差异，在回归式( 22) 中将前景理论效用值替换成期望效用理论效用值进行分析，具体回归式如下:
\begin{equation}
R_{i, t+1}=\alpha_{0}+\alpha_{1} P T_{i, t}^{r a n}+\sum_{j=1}^{n} \beta_{j} \text { Control } l_{j, i, t}^{r a n}  \qquad(21)\\
R_{i, t+1}=\alpha_{0}+\alpha_{1} E U_{i, t}^{r a n}+\sum_{j=1}^{n} \beta_{j} \text { Control }_{j, i, t}^{r a n}   \qquad(22)
\end{equation}

In [251]:
alldata_2=alldata_copy.dropna(subset=['mom','skew','pt','eu'])

res2_1=FM(alldata_2, dv=['Rt_1'], idv=['IVOL','Size','BM','Turnover','mom','skew'])
res2_2=FM(alldata_2, dv=['Rt_1'], idv=['pt','Size','BM','Turnover','mom','skew'])
res2_3=FM(alldata_2, dv=['Rt_1'], idv=['eu','Size','BM','Turnover','mom','skew'])

![title](png/4.png)

回归式( 20) 的结果表明，在加入了影响股票横截面收益率的其他控制变量后，特质波动率与预期收益率的负相关关系依然显著，我国股票市场上存在“特质波动率之谜”。此外，从控制变量的回归系数中可以发现，偏度、账面市值比和动量对应的回归系数不显著外，而流通市值和换手率会对预期收益率有显著的影响． 具体而言，小市值股票的预期收益率显著高于大市值股票的预期收益率; 价值股的预期收益率显著高于成长股的预期收益率; 随着换手率的增加，股票的预期收益率将会下降; 过去一段时间表现较好的股票，未来会有较高的预期收益率。

鉴于在Fama-MacBeth 回归中使用的是规模化处理后的变量，因此可以从多空组合自融资策略收益率的角度对各个解释变量的回归系数进行比较。其中，买入最低特质波动率组合同时卖空最高特质波动率组合的策略可以获得1.5%月平均收益率，次于基于流通市值的策略(4.3%) 和基于换手率的策略(2.3%) ，显著高于基于动量的策略(0.2%) 和基于BM 值的策略(0.2%)。这也进一步说明了最高特质波动率组合与最低特质波动率组合的预期收益率存在明显的差异．

对比式( 21) 和式( 22) 的回归结果，可以发现预期收益率与前景理论效用值之间的负相关关系显著，而与期望效用理论效用值之间的关系并不显著。
这说明投资者对股票历史收益率分布的衡量方式更接近前景理论的模型设定，前景理论中孤立效应、反射效应、损失厌恶、高估小概率事件等特性更符合实际投资者的特点，投资者会在前景理论的框架下评估风险，倾向于购买前景理论效用值高的股票，导致这部分股票的价格上升，预期收益率下降． 因此前景理论效用值越高的股票，预期收益率越低，而期望效用理论效用值与预期收益率的关系则不明显．

-------------

## 稳健性检验

**CGO指标替换**

除了前文使用的以参考点调整收益率来衡量股票未实现的资本利得外，Grinblatt 和Han［29］也定义了计算股票未实现的资本利得的方法，以衡
量股票当前价格相对于其参考点价格的偏离程度，即：
$$
C G O_{i, t}=\left(P_{i, t}-R P_{i, t}\right) / P_{i, t}
$$
其中 $P_{i, t}$ 为月内最后一个交易日的价格, $R P_{i, t}$ 为 对应的参考点价格, 参考点价格的定义为
$$
\begin{array}{l}
R P_{i, t}= \\
\quad \frac{1}{k} \sum_{n=1}^{T}\left[V_{i, t-n} \prod_{\tau=1}^{n-1}\left(1-V_{i, t-n+\tau}\right)\right] P_{i, t-n}
\end{array}
$$

其中Vi，t为股票的日度换手率，T 为计算周期内全部交易日的总个数，k 是一个常量，用来调整权重的大小以保证全部权重之和为1． 这里以每个交易日向前回
溯3年的全部换手率信息为基础，计算全部交易日的换手率加权平均价格作为参考点价格． 同样，当CGO ＜ 0 时，记为CL 组; 当CGO ＞ 0时，记为CG 组． 本文使用上述方法重复了前文的实证分析过程，实证结果依然稳健．

**参数设定**

首先，在计算前景理论效用值时，本文选择了被学术界广泛认可和使用的Tversky 和Kahneman［5］的参数校准结果，Baeberis 和Xiong［30］在研究中指出前景理论中的价值函数性质在解释资本市场中投资者行为的过程中起到更为重要的作用，因此本文在Tversky 和Kahneman［5］的参数设定基础上对价值函数中的α 和λ 进行调整，根据不同的参数组合计算对应的前景理论效用值，然后利用回归式( 21) 对整体市场进行Fama-Mac-Beth 回归检验，以考察在不同参数取值条下，前景理论效用值与期望收益之间的负相关关系是否稳健，表7 给出了不同参数组合条件下的回归系数与对应的t 统计量．
![title](png/5.png)

其次，在计算期望效用理论效用值时，将CRRA 型效用函数中参数θ 的取值设定为0． 5，为了保证研究结论的稳健性，同样需要对参数θ 的敏感性进行分析． 在设定参数θ 取值范围的基础上，根据给定的θ 值计算对应的期望效用理论效用值，然后利用回归式( 22) 对整体市场进行Fama-MacBeth 回归检验，以考察在不同参数取值条件下，期望理论效用值与期望收益不相关的结论是否稳健.
![title](png/6.png)

参数敏感性的检验结果表明: 1) 无论前景理论效用函数的参数取值如何变化，前景理论效用值与期望收益间的负相关关系均是稳健的; 2) 除了在θ 取值0． 9 时得到的期望效用理论效用值的回归系数在10%的显著性水平下显著以外，其他参数条件下期望效用理论效用值与预期收益率无关． 这也进一步说明，相比期望效用理论，前景理论的独特设定能更准确地刻画现实投资者的风险偏好，基于前景理论视角的投资者行为分析能更好地解释金融市场中的异象．